<a href="https://colab.research.google.com/github/girish445ai/Convolutional_Neural_Networks/blob/main/Convolutional_Neural_Networks_PART_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install wandb

In [10]:
import numpy as np
import keras 
import tensorflow as tf
import pandas as pd
import wandb
import matplotlib.pyplot as plt
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from wandb.keras import WandbCallback

In [11]:
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip

train_datapath='inaturalist_12K/train/'
test_datapath='inaturalist_12K/val/'
categories=['Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia']

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3639M  100 3639M    0     0   145M      0  0:00:25  0:00:25 --:--:--  197M
Archive:  nature_12K.zip
replace inaturalist_12K/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Part A Q2

In [4]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = train_datapath,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = True,
                      seed = 42,
                      validation_split = 0.2,
                      subset = 'training')

val_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = train_datapath,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      batch_size = 32,
                      image_size = (256, 256),
                      color_mode = 'rgb',
                      shuffle = True,
                      seed = 42,
                      validation_split = 0.2,
                      subset = 'validation')
test_data = tf.keras.preprocessing.image_dataset_from_directory(
                      directory = test_datapath,
                      labels = 'inferred',  
                      label_mode = 'categorical',
                      color_mode = 'rgb',
                      batch_size = 32,
                      image_size = (256, 256),
                      shuffle = False,
                      seed = 17,
                      validation_split = 0)

Found 9999 files belonging to 10 classes.
Using 8000 files for training.
Found 9999 files belonging to 10 classes.
Using 1999 files for validation.


In [5]:
# 20% validation data
len_train, len_val = len(train_data), len(val_data)
train_data = train_data.take(int(0.2*len_train))
val_data = val_data.take(int(0.2*len_val))

In [6]:
train_data_aug = ImageDataGenerator(
                            rescale = None,
                            rotation_range = 20,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            horizontal_flip=True,
                            data_format = 'channels_last',
                            validation_split = 0.2)        

val_data_aug = ImageDataGenerator(validation_split = 0.2) 

train_aug_gen = train_data_aug.flow_from_directory(train_datapath, shuffle = True, \
                                                         seed = 42, subset = 'training')
val_aug_gen = val_data_aug.flow_from_directory(train_datapath, shuffle = True, \
                                                     seed = 42, subset = 'validation')

train_aug_data = tf.data.Dataset.from_generator(
                    lambda: train_aug_gen,
                    output_types = (tf.float32, tf.float32),
                    output_shapes = ([None, 256, 256, 3], [None, 10]))

val_aug_data = tf.data.Dataset.from_generator(
                  lambda: val_aug_gen,
                  output_types = (tf.float32, tf.float32),
                  output_shapes = ([None, 256, 256, 3], [None, 10]))

Found 8000 images belonging to 10 classes.
Found 1999 images belonging to 10 classes.


In [7]:
# 20% validation data
len_train_aug, len_val_aug = len(train_aug_gen), len(val_aug_gen)
train_aug_data = train_aug_data.take(int(0.2*len_train_aug))
val_aug_data = val_aug_data.take(int(0.2*len_val_aug))

In [8]:
wandb.init()

sweep_config = {
    'method': 'random', #grid, random, bayes
    'metric': {
      'name': 'val_acc',
      'goal': 'maximize'
      },
    'parameters': {
        'activation': {
            'values': ['relu','leaky_relu']
        },
        'kernel_size': {
            'values': [3, 5, 7] 
        },
                
        'filter_org': {
            'values': [0.5, 1, 2]
        },

        'dense_size': {
            'values': [32, 64, 128]
        },

        'first_layer_filters': {
            'values': [32, 64]
        },

        'conv_layers': {
            'values': [5]
        },

        'dropout': {
            'values': [0.2, 0.3]
        },

        'num_epochs': {
            'values': [10]
        },

        'batch_norm': {
            'values': [True, False]
        },

        'optimizer': {
            'values': ['adam']
        },

        'data_aug': {
            'values': [True, False]
        },
    }
}

wandb: Currently logged in as: jyothiraditya (use `wandb login --relogin` to force relogin)


Problem at: <ipython-input-8-5b1fac671385> 1 <module>


KeyboardInterrupt: ignored

In [ ]:
def cnn_train(config):
    with wandb.init(config=config):
        config = wandb.init().config
        wandb.run.name = 'firstLayerFilters_{}_filterOrg_{}_dataAug_{}_batchNorm_{}_dropout_{}_kerSize_{}_denseSize_{}'.format(config.first_layer_filters, config.filter_org, config.data_aug, config.batch_norm, config.dropout, config.kernel_size, config.dense_size)
        
        inputs = tf.keras.Input(shape = (256, 256, 3))
        m = tf.keras.layers.Rescaling(scale = 1.0/255)(inputs)
        filter_sizes = [int(config.first_layer_filters*(config.filter_org**layer_num)) for layer_num in range(config.conv_layers)]
        ker_size = config.kernel_size

        # Adding layers
        for layer_num in range(config.conv_layers):
            m = layers.Conv2D(filters = filter_sizes[layer_num], kernel_size = (ker_size, ker_size))(m)
            if config.batch_norm == 1:
                mod = layers.BatchNormalization(axis = -1)(m)
            m = layers.Activation(config.activation)(m)
            m = layers.MaxPooling2D(pool_size = (2, 2))(m)            
                
        # Dense Layer
        m = layers.Flatten()(m)
        m = layers.Dense(config.dense_size)(m)
        if config.batch_norm:
            m = layers.BatchNormalization(axis = -1)(m)
        m = layers.Activation(config.activation)(m)
        if config.dropout > 0:
            m = layers.Dropout(rate = config.dropout)(m)        

        # Output Layer
        outputs = layers.Dense(10, activation ='softmax')(m)
        model = tf.keras.Model(inputs = inputs, outputs = outputs)

        sweep_val_data = val_data # In any case, validation data is not augmented
        if config.data_aug == 1:
          sweep_train_data = train_aug_data
        else:
          sweep_train_data = train_data


        model.compile(optimizer=config.optimizer,
                      loss = tf.keras.losses.CategoricalCrossentropy(name = 'loss'),
                      metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'acc')])
        
        model_hist = model.fit(sweep_train_data, epochs = config.num_epochs,
                               validation_data = sweep_val_data, 
                               callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5), wandb.keras.WandbCallback()])

In [ ]:
wandb.init(config = sweep_config)
config = wandb.config
sweep_id = wandb.sweep(config, project = 'assgn_jy2')
wandb.agent(sweep_id, function=cnn_train(config))

#Part A Q4

#4a

In [ ]:
best = {
              'first_layer_filters': 32,
              'filter_org': 2, 
              'data_aug': True,
              'batch_norm': True, 
              'dropout': 0.2,
              'kernel_size': 3,
              'dense_size': 128,
              'activation': 'relu',
              'num_epochs': 10, 
              'optimizer': 'adam',
              'conv_layers': 5
          }

In [ ]:
inputs = tf.keras.Input(shape = (256, 256, 3))
m = tf.keras.layers.Rescaling(scale = 1.0/255)(inputs)
filter_sizes = [int(best['first_layer_filters']*(best['filter_org']**layer_num)) 
                for layer_num in range(best['conv_layers'])]
ker_size = best['kernel_size']

for layer_num in range(best['conv_layers']):
    m = layers.Conv2D(filters = filter_sizes[layer_num], kernel_size = (ker_size, ker_size))(m)
    if best['batch_norm'] == 1:
        m = layers.BatchNormalization(axis = -1)(m)
    m = layers.Activation(best['activation'])(m)
    m = layers.MaxPooling2D(pool_size = (2, 2))(m)            
               
m = layers.Flatten()(m)
m = layers.Dense(best['dense_size'])(m)
if best['batch_norm'] == True:
    m = layers.BatchNormalization(axis = -1)(m)
m = layers.Activation(best['activation'])(m)
if best['dropout'] > 0:
    m = layers.Dropout(rate = best['dropout'])(m)        

outputs = layers.Dense(10, activation ='softmax')(m)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
        
model.compile(optimizer = best['optimizer'], loss = tf.keras.losses.CategoricalCrossentropy(name = 'loss'), metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'acc')])
if optimal['data_aug'] == True:
  model_hist = model.fit(train_aug_data, epochs = optimal['num_epochs'],
                       callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'acc', patience = 5)])
else:
  model_hist = model.fit(train_data, epochs = optimal['num_epochs'],
                       callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'acc', patience = 5)])

In [ ]:
run = wandb.init(config=optimal, project="assgn2", entity="jyothiraditya")
config = wandb.config
test_loss, test_acc = model.evaluate(test_data)
wandb.log({"test_loss": test_loss, "test_accuracy": test_acc})

#4b

In [ ]:
plt.figure(figsize=(15, 60))
images, labels = train_data.as_numpy_iterator().next()
for i in range(30):
    ax = plt.subplot(10, 3, i + 1)
    plt.imshow(images[i].astype("uint8"))
    predicted_cat = categories[np.argmax(model.predict(np.array(images[i,:,:,:]).reshape(1, 256, 256, 3)))]
    title_obj = plt.title('True class: {}\nPredicted class : {}'.format(categories[np.where(labels[i] == 1)[0][0]],predicted_cat))
    plt.setp(title_obj, color='r')
    if categories[np.where(labels[i] == 1)[0][0]] == predicted_cat:
      plt.setp(title_obj, color='g')
    plt.axis("off")
    
wandb.log({'img': [wandb.Image(ax)]})